# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости.

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

### Изучение данных

In [1]:
!pip3 install catboost
!pip install lightgbm
!pip install scikit-learn==1.1.3

# библиотеки
import pandas as pd
import numpy as np
import lightgbm

# модели
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.dummy import DummyRegressor

# метрика
from sklearn.metrics import mean_squared_error

# расчеты
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

import warnings
warnings.filterwarnings("ignore")

# константы
RANDOM_STATE = 123

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 35.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [2]:
# Откроем файл
try:
    data = pd.read_csv('autos.csv')

except:
    from google.colab import files
    uploaded = files.upload()

Saving autos.csv to autos.csv


In [4]:
# Откроем файл
import io
data = pd.read_csv(io.BytesIO(uploaded['autos.csv']))

# Сделаем копию датасета
df = data.copy()

# Изучим основную информацию о данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [5]:
# Изучим основную информацию о данных
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [6]:
# Изучим основную информацию о данных
df.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


Посмотрим список уникальных значений в интересующих нас столбцах.

In [7]:
# Отберем столбцы в отдельный список
b = ['VehicleType', 'RegistrationYear', 'Gearbox', 'Power', 'Model', 'RegistrationMonth', 'FuelType', 'Brand', 'Repaired']

# Выведем список уникальных значений
for i in df[b]:
    print(i)
    print(df[i].sort_values().unique())

VehicleType
['bus' 'convertible' 'coupe' 'other' 'sedan' 'small' 'suv' 'wagon' nan]
RegistrationYear
[1000 1001 1039 1111 1200 1234 1253 1255 1300 1400 1500 1600 1602 1688
 1800 1910 1915 1919 1920 1923 1925 1927 1928 1929 1930 1931 1932 1933
 1934 1935 1936 1937 1938 1940 1941 1942 1943 1944 1945 1946 1947 1948
 1949 1950 1951 1952 1953 1954 1955 1956 1957 1958 1959 1960 1961 1962
 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 1975 1976
 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990
 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004
 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018
 2019 2066 2200 2222 2290 2500 2800 2900 3000 3200 3500 3700 3800 4000
 4100 4500 4800 5000 5300 5555 5600 5900 5911 6000 6500 7000 7100 7500
 7800 8000 8200 8455 8500 8888 9000 9229 9450 9996 9999]
Gearbox
['auto' 'manual' nan]
Power
[    0     1     2     3     4     5     6     7     8     9    10    11
    12    13    14    1

**Вывод:** видим пропуски в данных, во многих столбцах аномальные значения (например, год регистрации авто и мощность), некоторые столбцы вообще не содержат значимой информации (количество фотографий - везде "0"). Также требуется переименовать столбцы датасета по правилам записи. С типом данных всё в порядке, кроме столбцов с датами. Однако, эти столбы неинформативны ("дата скачивания анкеты из базы", "дата создания анкеты" и "дата последней активности пользователя"), поэтому удалим их.

### Предобработка данных

#### Дубликаты

In [8]:
# Проверим датасет на наличие явных дубликатов
print(df.duplicated().sum())

4


In [9]:
# Удалим явные дубликаты
df = df.drop_duplicates(keep='first')

В столбце "fuel_type" в списке уникальных значений мы видим дублирование. Petrol и gasoline	- это бензин. Заменим все на petrol.

In [10]:
# Заменим gasoline на petrol
df.loc[df['FuelType'] == 'gasoline', 'FuelType'] = 'petrol'

#### Названия и количество столбцов

Информация в некоторых столбцах никак не влияет на цену автомобиля, удалим эти столбцы из датасета - 'DateCrawled', 'DateCreated', 'NumberOfPictures', 'LastSeen', 'PostalCode'.

Столбец 'RegistrationMonth' (месяц регистрации автомобиля) тоже не несет в себе значимой ценности, при покупке б/у автомобиля более важными параметрами являются год регистрации и пробег. Также в списке уникальных значений видим явные аномалии - нулевой месяц. Также удалим этот столбец.

In [11]:
# Удалим столбцы, которые не несут в себе значимой информации
df = df.drop(['DateCrawled', 'DateCreated', 'NumberOfPictures', 'LastSeen', 'PostalCode', 'RegistrationMonth'], axis=1)
df.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,Repaired
0,480,NaN,1993,manual,0,golf,150000,petrol,volkswagen,NaN
1,18300,coupe,2011,manual,190,NaN,125000,petrol,audi,yes
2,9800,suv,2004,auto,163,grand,125000,petrol,jeep,NaN
3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,petrol,skoda,no


In [12]:
# Переименуем столбцы
df.columns = ['price', 'vehicle_type', 'registration_year', 'gearbox', 'power', 'model', 'kilometer', 'fuel_type', 'brand', 'repaired']
df.head()

,price,vehicle_type,registration_year,gearbox,power,model,kilometer,fuel_type,brand,repaired
0,480,NaN,1993,manual,0,golf,150000,petrol,volkswagen,NaN
1,18300,coupe,2011,manual,190,NaN,125000,petrol,audi,yes
2,9800,suv,2004,auto,163,grand,125000,petrol,jeep,NaN
3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,petrol,skoda,no


#### Пропуски

In [13]:
# Количество пропусков
df.isna().sum()

price                    0
vehicle_type         37490
registration_year        0
gearbox              19833
power                    0
model                19705
kilometer                0
fuel_type            32895
brand                    0
repaired             71154
dtype: int64

Мы видим достаточно большое количество пропусков в нескольких столбцах (около 5-20% от общего кол-ва данных). Поскольку мы решаем задачу прогнозирования, то нам важно сохранить как можно больше информации.
* Пропуски в столбце 'vehicle_type' заполним заглушкой, поскольку одна и та же модель автомобиля может иметь разные типы кузова, эти данные невозможно корректно восстановить. При этом тип кузова лишь опосредованно влияет на цену автомобиля, поскольку он зависит от марки и модели авто.
* Пропуски в столбце 'model'. Корректно восстановить данные по другим столбцам невозможно. При этом модель автомобиля существенно влияет на его стоимость. Пропуски составляют 5,5% от общего количества данных, удалим эти строки.
* Пропуски в столбцах 'gearbox' и 'fuel_type' заполним модой по модели автомобиля.
* Пропуски в столбце 'repaired' заполним заглушкой, эти данные невозможно корректно восстановить.

In [14]:
# Заглушка в столбцах 'repaired' и 'vehicle_type'
df.loc[df['repaired'].isna(), 'repaired'] = 'unknown'
df.loc[df['vehicle_type'].isna(), 'vehicle_type'] = 'unknown'

In [15]:
# Удалим пропуски в столбце 'model'
df = df.dropna(subset=['model'])

In [16]:
# Заполним пропуски в столбцах 'gearbox' и 'fuel_type'
grp = df.groupby(['model'])
df['gearbox'] = grp.gearbox.apply(lambda x: x.fillna(x.mode()[0]))
df['fuel_type'] = grp.gearbox.apply(lambda x: x.fillna(x.mode()[0]))

In [17]:
# Проверим результат
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 334660 entries, 0 to 354368
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   price              334660 non-null  int64 
 1   vehicle_type       334660 non-null  object
 2   registration_year  334660 non-null  int64 
 3   gearbox            334660 non-null  object
 4   power              334660 non-null  int64 
 5   model              334660 non-null  object
 6   kilometer          334660 non-null  int64 
 7   fuel_type          334660 non-null  object
 8   brand              334660 non-null  object
 9   repaired           334660 non-null  object
dtypes: int64(4), object(6)
memory usage: 28.1+ MB


#### Аномалии

В датасете достаточно много аномальных числовых значений, рассмотрим все столбцы попорядку.
* **price** - значения от 0 до 20 000 евро. Верхняя граница выглядит правдоподобно. Значения меньше 10 удалим из датасета.
* **registration_year** - значения от 1000 до 9999. Нижнюю границу ограничим значением 1900, верхнюю - 2016 г., поскольку данные в датасете представлены именно за 2016 год. Корректно восстановить неверную информацию у нас не получится, удалим все аномалии.
* **power** - значения от 0 до 20 000 л.с. Нижнюю границу ограничим уровнем 10 л.с., менее мощные автомобили бывают, но это уже экзотика. Верхняя граница - 1600 л.с., по состоянию на 2016 год это был самый мощный автомобиль (на данный момент есть мощнее).
* **kilometer** - все значения выглядят правдоподобно.

In [18]:
# Удалим аномальные значения из датасета
df = df.query('price >= 10')
df = df.query('registration_year >= 1900 and registration_year <= 2016')
df = df.query('power >= 10 and power <= 1600')

# Проверим результат
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284866 entries, 2 to 354368
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   price              284866 non-null  int64 
 1   vehicle_type       284866 non-null  object
 2   registration_year  284866 non-null  int64 
 3   gearbox            284866 non-null  object
 4   power              284866 non-null  int64 
 5   model              284866 non-null  object
 6   kilometer          284866 non-null  int64 
 7   fuel_type          284866 non-null  object
 8   brand              284866 non-null  object
 9   repaired           284866 non-null  object
dtypes: int64(4), object(6)
memory usage: 23.9+ MB


#### Работа со столбцами "модель" и "бренд"

В столбцах Brand и Model находятся суммарно несколько десятков уникальных значений, что при кодировании OHE приведет к образованию большого количества столбцов.

In [19]:
# Посмотрим наиболее редкие модели
df['model'].value_counts().tail(25)

v60                   39
croma                 37
move                  36
145                   34
terios                32
lodgy                 30
delta                 27
b_max                 25
9000                  23
musa                  21
charade               20
materia               17
200                   16
q3                    13
kappa                 13
gl                    12
amarok                12
elefantino             7
serie_2                5
kalina                 5
i3                     5
samara                 5
serie_3                3
rangerover             2
range_rover_evoque     2
Name: model, dtype: int64

Удалим наиболее редкие модели (менее 30 упоминаний) из датасета.

In [20]:
# Получим список редких моделей
model_drop = (df['model'].value_counts().tail(20)).index.to_list()

# Удалим редкие модели
df = df.query("model != ['lodgy', 'delta', 'b_max', '9000', 'musa', 'charade', 'materia', '200', \
 'q3', 'kappa', 'gl', 'amarok', 'elefantino', 'i3', 'kalina', 'serie_2', 'samara', 'serie_3', 'range_rover_evoque', 'rangerover']")

In [21]:
# Посмотрим наиболее редкие бренды
df['brand'].value_counts().tail(10)

daihatsu      563
jeep          562
land_rover    445
saab          439
jaguar        431
daewoo        385
rover         318
lancia        298
trabant       261
lada          141
Name: brand, dtype: int64

Наиболее редкий бренд - Лада. Однако, количество упоминаний больше 100, поэтому оставим все данные.

In [22]:
# Проверим общее количество удаленных данных
print('Всего удалено данных из датасета:', 1 - len(df) / len(data))

Всего удалено данных из датасета: 0.19687388005158468


**Вывод:** мы подготовили датасет для дальнейшей работы, заполнили пропуски, удалили аномалии, в сумме удалено 19,7% некачественных данных.

### Подготовка признаков

Разделим исходные данные на обучающую, валидационную и тестовую выборки. Размеры тестового и валидационного наборов обычно равны. Таким образом, нам необходимо разбить данные в соотношении 3:1:1. Или 60% - обучающая выборка, и по 20% - тестовая и валидационная.

In [23]:
# Выделим целевой признак
features = df.drop(['price'], axis=1)
target = df['price']

# Разделим данные на 2 выборки: 40% тестовая и валидационная вместе взятые, и 60% обучающая
X_train, X_test, y_train, y_test = \
    train_test_split(features, target, test_size=0.4, random_state=RANDOM_STATE)

# Разделим 40% на тестовую и валидационную выборки
X_test, X_valid, y_test, y_valid = \
    train_test_split(X_test, y_test, test_size=0.5, random_state=RANDOM_STATE)

# Проверим размер получившихся выборок
print(X_train.shape)
print(X_test.shape)
X_valid.shape

(170761, 9)
(56921, 9)


(56921, 9)

Часть признаков в датасете является категориальными. Некоторые модели умеют работать с такими признаками без предварительной подготовки (CatBoost), но для других моделей необходимо кодирование категориальных признаков.

In [24]:
# Составим списки категориальных и численных признаков
category_features = X_train.select_dtypes(include='object').columns.to_list()
display(category_features)
numeric_features = X_train.select_dtypes(include='int64').columns.to_list()
display(numeric_features)

['vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand', 'repaired']

['registration_year', 'power', 'kilometer']

## Обучение моделей

Выполним кодирование категориальных признаков методом OHE и стандартизируем все численные признаки с помощью StandardScaler(). Объединим эти действия через make_column_transformer. Трансформировать необходимо как обучающую, так и валидационную выборку.

In [25]:
X_train_linear = X_train.copy()
X_valid_linear = X_valid.copy()

In [26]:
# Задаем параметры для make_column_transformer
col_transformer_linear = make_column_transformer(
    (
        OneHotEncoder(drop='first', handle_unknown='ignore'),
        category_features
    ),
    (
        StandardScaler(),
        numeric_features
    ),
    remainder='passthrough',
    verbose_feature_names_out=False
)

# Кодируем и масштабируем тренировочную выборку
X_train_linear = pd.DataFrame.sparse.from_spmatrix(
    col_transformer_linear.fit_transform(X_train_linear),
    columns=col_transformer_linear.get_feature_names_out()
)

# Смотрим результат
X_train_linear.head()

,vehicle_type_convertible,vehicle_type_coupe,vehicle_type_other,vehicle_type_sedan,vehicle_type_small,vehicle_type_suv,vehicle_type_unknown,vehicle_type_wagon,gearbox_manual,model_145,...,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,repaired_unknown,repaired_yes,registration_year,power,kilometer
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.110109,-0.189814,0.593035
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.639333,0.490631,0.593035
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.302219,-0.530036,0.593035
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.168557,-0.972325,-1.862062
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.929921,-0.325903,0.593035


In [27]:
# Кодируем и масштабируем валидационную выборку
X_valid_linear = pd.DataFrame.sparse.from_spmatrix(
    col_transformer_linear.transform(X_valid_linear),
    columns=col_transformer_linear.get_feature_names_out()
)

X_valid_linear.head()

,vehicle_type_convertible,vehicle_type_coupe,vehicle_type_other,vehicle_type_sedan,vehicle_type_small,vehicle_type_suv,vehicle_type_unknown,vehicle_type_wagon,gearbox_manual,model_145,...,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,repaired_unknown,repaired_yes,registration_year,power,kilometer
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.772995,-0.104758,-0.770908
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-2.342249,-1.125425,-0.088936
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.459145,-1.142436,-2.134851
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.011631,1.000964,0.593035
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.423960,1.341187,-1.862062


### Линейная регрессия

In [28]:
%%time

# Обучим модель
model_linear = LinearRegression()
model_linear.fit(X_train_linear, y_train)

CPU times: user 3.43 s, sys: 2.49 s, total: 5.92 s
Wall time: 3.18 s


LinearRegression()

In [29]:
%%time
# Получим предсказания
predictions_valid_linear = model_linear.predict(X_valid_linear)

# Рассчитаем и выведем RMSE
result_linear = mean_squared_error(y_valid, predictions_valid_linear)**0.5
print('RMSE на валидационной выборке:', result_linear.round(3))

RMSE на валидационной выборке: 2629.358
CPU times: user 40.5 ms, sys: 34.1 ms, total: 74.6 ms
Wall time: 40.9 ms


### Случайный лес

В данном случае нам подойдут уже преобразованные ранее признаки, будем использовать их. Лучшие параметры подберем через RandomizedSearchCV.

In [30]:
%%time

# Обучим модель
model_rf = RandomForestRegressor(random_state=RANDOM_STATE, n_estimators=25, max_depth=12)
model_rf.fit(X_train_linear, y_train)

CPU times: user 1min 24s, sys: 29.2 ms, total: 1min 24s
Wall time: 1min 32s


RandomForestRegressor(max_depth=12, n_estimators=25, random_state=123)

In [31]:
%%time

# Получим предсказания
predictions_valid_rf = model_rf.predict(X_valid_linear)

# Рассчитаем и выведем RMSE
result_rf = mean_squared_error(y_valid, predictions_valid_rf)**0.5
print('RMSE на валидационной выборке:', result_rf.round(3))

RMSE на валидационной выборке: 1772.616
CPU times: user 256 ms, sys: 747 µs, total: 256 ms
Wall time: 270 ms


### Ridge

In [32]:
# Подберем гиперпараметры
model_rid = Ridge(random_state=RANDOM_STATE)

parameters_rid = {'alpha': np.arange(0, 0.11, 0.02)}

grid_ridge = GridSearchCV(model_rid, parameters_rid, scoring='neg_root_mean_squared_error', n_jobs=-1)
grid_ridge.fit(X_train_linear, y_train)

grid_ridge.best_params_

{'alpha': 0.06}

In [33]:
%%time

# Обучим модель
model_rid = Ridge(random_state=RANDOM_STATE, alpha=0.00)
model_rid.fit(X_train_linear, y_train)

CPU times: user 558 ms, sys: 393 ms, total: 951 ms
Wall time: 506 ms


Ridge(alpha=0.0, random_state=123)

In [34]:
%%time

# Получим предсказания
predictions_valid_rid = model_rid.predict(X_valid_linear)

# Рассчитаем и выведем RMSE
result_rid = mean_squared_error(y_valid, predictions_valid_rid)**0.5
print('RMSE на валидационной выборке:', result_rid.round(3))

RMSE на валидационной выборке: 2630.158
CPU times: user 29.5 ms, sys: 19.9 ms, total: 49.5 ms
Wall time: 42.9 ms


### CatBoost

В модель CatBoost передаем признаки без кодирования и масштабирования (X_train).

In [35]:
%%time

# Обучим модель
model_cat = CatBoostRegressor(loss_function='RMSE',
                              cat_features=category_features,
                              iterations=80,
                              depth = 11,
                              random_state=RANDOM_STATE
                             )
model_cat.fit(X_train, y_train, verbose=20)

Learning rate set to 0.5
0:	learn: 3074.2262740	total: 324ms	remaining: 25.6s
20:	learn: 1603.6386657	total: 4.04s	remaining: 11.3s
40:	learn: 1489.8824178	total: 6.75s	remaining: 6.42s
60:	learn: 1432.8321566	total: 9s	remaining: 2.8s
79:	learn: 1398.0394967	total: 11.1s	remaining: 0us
CPU times: user 18.2 s, sys: 396 ms, total: 18.6 s
Wall time: 11.8 s


In [36]:
%%time

# Получим предсказания
predictions_valid_cat = model_cat.predict(X_valid)

# Рассчитаем и выведем RMSE
result_cat = mean_squared_error(y_valid, predictions_valid_cat)**0.5
print('RMSE на валидационной выборке:', result_cat.round(3))

RMSE на валидационной выборке: 1571.279
CPU times: user 131 ms, sys: 1.97 ms, total: 133 ms
Wall time: 102 ms


### LightGBM

Для модели LightGBM передаем признаки без кодирования и масштабирования, но предварительно все категориальные столбцы нужно перевести в тип "category".

In [37]:
X_train_lgbm = X_train.copy()
X_valid_lgbm = X_valid.copy()
X_train_lgbm[category_features] = X_train_lgbm[category_features].astype('category')
X_valid_lgbm[category_features] = X_valid_lgbm[category_features].astype('category')

In [42]:
%%time

# Обучим модель
model_lgbm = LGBMRegressor(learning_rate = 0.1778279410038923,
                           n_estimators = 60,
                           num_leaves = 40,
                           random_state=RANDOM_STATE
                             )
model_lgbm.fit(X_train_lgbm, y_train)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013602 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 623
[LightGBM] [Info] Number of data points in the train set: 170761, number of used features: 9
[LightGBM] [Info] Start training from score 4903.684583
CPU times: user 1.07 s, sys: 6.12 ms, total: 1.07 s
Wall time: 1.08 s


LGBMRegressor(learning_rate=0.1778279410038923, n_estimators=60, num_leaves=40,
              random_state=123)

In [43]:
%%time

# Получим предсказания
predictions_valid_lgbm = model_lgbm.predict(X_valid_lgbm)

# Рассчитаем и выведем RMSE
result_lgbm = mean_squared_error(y_valid, predictions_valid_lgbm)**0.5
print('RMSE на валидационной выборке:', result_lgbm.round(3))

RMSE на валидационной выборке: 1581.194
CPU times: user 311 ms, sys: 3.74 ms, total: 315 ms
Wall time: 316 ms


## Анализ моделей

### Сравнительный анализ

Сведем все результаты в одну таблицу и проанализируем качество моделей.

In [44]:
# Итоговая таблица
result = {'RMSE': [result_linear, result_rf, result_rid, result_cat, result_lgbm],
          'Время обучения': ['25,7 s', '1 m 04 s', '2,99 s', '9,14 s', '3,12 s'],
          'Время предсказания': ['203 ms', '172 ms', '104 ms', '78,5 ms', '396 ms']}
df_result = pd.DataFrame(result, index =['LinearRegression', 'RandomForestRegressor', 'Ridge', 'CatBoost', 'LightGBM'])
df_result

,RMSE,Время обучения,Время предсказания
LinearRegression,2629.357626,"25,7 s",203 ms
RandomForestRegressor,1772.616234,1 m 04 s,172 ms
Ridge,2630.158129,"2,99 s",104 ms
CatBoost,1571.279036,"9,14 s","78,5 ms"
LightGBM,1581.193552,"3,12 s",396 ms


Заказчику важны следующие критерии:
*	качество предсказания;
*	время обучения модели;
*	время предсказания модели.

Как видно из сводной таблицы, наилучшее качество (наименьшее значение RMSE) на валидационной выборке показала модель CatBoost, чуть хуже качество у RandomForestRegressor и LightGBM.
При этом RandomForestRegressor существенно проигрывает в скорости обучения. Время предсказания несколько невелико во всех моделях, что им можно пренебречь в данном случае.

Поскольку качество модели CatBoost выше, чем у LightGBM, а время обучения при этом вполне приемлемое, выберем её в качестве лучшей модели и проверим ее качество на тестовой выборке.

### Проверка лучшей модели на тестовой выборке

Поскольку в модель CatBoost передаются признаки без кодирования и масштабирования, то дополнительных преобразований для тестовой выборки не требуется.

In [45]:
%%time

# Получим предсказания
predictions_test_cat = model_cat.predict(X_test)

# Рассчитаем и выведем RMSE
result_test_cat = mean_squared_error(y_test, predictions_test_cat)**0.5
print('RMSE на тестовой выборке:', result_test_cat.round(3))

RMSE на тестовой выборке: 1580.267
CPU times: user 138 ms, sys: 2.86 ms, total: 141 ms
Wall time: 113 ms


Итак, на тестовой выборке мы получили значение RMSE в размере 1562, что ниже требуемого Заказчиком. **Лучшая модель - CatBoost со следующими гиперпараметрами: {'depth': 11, 'iterations': 80}, время обучения модели - 9 сек, время предсказания на тестовой выборке - 113 мс.**

В заключение сравним результаты нашей модели с результатами константной модели. В нашем случае подойдет DummyRegressor.

In [46]:
# Получим предсказания константной модели
model_dummy = DummyRegressor(strategy="mean")
model_dummy.fit(X_train, y_train)
predictions_test_dummy = model_dummy.predict(X_test)
result_test_dummy = mean_squared_error(y_test, predictions_test_dummy)**0.5
print('RMSE dummy:', result_test_dummy.round(3))

RMSE dummy: 4615.573


## Итоговый вывод

- в данном проекте было обучено 5 моделей;
- качество моделей измерялось метрикой RMSE, а также временем обучения и временем предсказания;
- наименьшее значение RMSE показала модель CatBoost с гиперпараметрами: depth - 11, iterations - 80, однако по времени обучения модель проигрывает LightGBM;
- модель LightGBM существенно быстрее обучается и показывает достаточно хорошие результаты метрики RMSE;
- учитывая приоритеты, расставленные заказчиком, в качестве лучшей модели выбрана модель CatBoost. Однако, если время обучения важнее качества модели, то рекомендуется обратиться к модели LightGBM;
- лучшая модель была проверена на тестовой выборке и показала значение RMSE в размере 1580;
- в сравнении с константной моделью выбранная модель показывает существенно более хороший результат (RMSE константной модели = 4613).  